In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [2]:
# Load the dataset
data = pd.read_csv("dataset.csv")

# Preprocessing
# Encode the label column
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])

# Separate features and target
X = data.drop(columns=['label']).values
y = data['label'].values

# Normalize the input features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert labels to categorical (one-hot encoding)
y = to_categorical(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
import h5py

# Build and train the ANN model (as in your code)
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),  # Hidden layer 1
    Dense(32, activation='relu'),  # Hidden layer 2
    Dense(y_train.shape[1], activation='softmax')  # Output layer
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Save the trained model
model.save("model.h5")

# Append the LabelEncoder and scaler attributes to the same `.h5` file
with h5py.File("model.h5", "a") as f:
    # Save LabelEncoder classes
    f.create_dataset("label_encoder_classes", data=label_encoder.classes_)
    
    # Save scaler mean and scale
    f.create_dataset("scaler_mean", data=scaler.mean_)
    f.create_dataset("scaler_scale", data=scaler.scale_)

Epoch 1/50


c:\Users\harsh\OneDrive\Desktop\Semester 7\Internet of Things (CS539)\Project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1587 - loss: 2.9917 - val_accuracy: 0.4517 - val_loss: 2.4149
Epoch 2/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5659 - loss: 2.1082 - val_accuracy: 0.7330 - val_loss: 1.3662
Epoch 3/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7866 - loss: 1.1013 - val_accuracy: 0.8523 - val_loss: 0.7139
Epoch 4/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8899 - loss: 0.5945 - val_accuracy: 0.8977 - val_loss: 0.4712
Epoch 5/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9328 - loss: 0.3867 - val_accuracy: 0.9148 - val_loss: 0.3497
Epoch 6/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9498 - loss: 0.2820 - val_accuracy: 0.9261 - val_loss: 0.2737
Epoch 7/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9558 - loss: 0.2296 - val_accuracy: 0.9489 - val_loss: 0.2252
Epoch 8/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9673 - loss: 0.1756 - val_accuracy: 0.9489 - val_loss: 0.1902
Epo

Test Accuracy: 0.97


In [4]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load the trained model
model = load_model("model.h5")

# Load preprocessing objects (LabelEncoder and scaler) from the `.h5` file
with h5py.File("model.h5", "r") as f:
    # Load LabelEncoder classes
    label_encoder = LabelEncoder()
    label_encoder.classes_ = f["label_encoder_classes"][:]
    
    # Load scaler mean and scale
    scaler = StandardScaler()
    scaler.mean_ = f["scaler_mean"][:]
    scaler.scale_ = f["scaler_scale"][:]

# Example new data (N, P, K, temperature, humidity, pH, rainfall)
new_data = np.array([[100, 30, 40, 26.5, 75.0, 6.5, 120]])

# Preprocess the input data (scale it)
new_data_scaled = scaler.transform(new_data)

# # Make predictions
predictions = model.predict(new_data_scaled)

# Get the indices of the top 3 probabilities
top_3_indices = np.argsort(predictions, axis=1)[:, -3:][:, ::-1]  # Sort and reverse to get top 3

# Decode the top 3 indices into crop labels and convert byte strings to regular strings
top_3_labels = [label.decode('utf-8') for label in label_encoder.inverse_transform(top_3_indices[0])]

# Get the corresponding probabilities for the top 3 labels
top_3_probabilities = predictions[0, top_3_indices[0]]

# Print the results
for i, (label, prob) in enumerate(zip(top_3_labels, top_3_probabilities)):
    print(f"Rank {i+1}: {label} (Probability: {prob:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Rank 1: coffee (Probability: 0.73)
Rank 2: jute (Probability: 0.23)
Rank 3: watermelon (Probability: 0.02)
